In [1]:
#default_exp psm_frag_reader.maxquant_frag_reader

In [2]:
#export
import pandas as pd
import numpy as np

from alphabase.peptide.fragment import (
    init_fragment_by_precursor_dataframe
)
from alphabase.io.psm_reader.maxquant_reader import (
    MaxQuantReader
)

from alphadeep.psm_frag_reader.psm_frag_reader import (
    PSMReader_w_FragBase,
    psm_w_frag_reader_provider
)

class MaxQuantMSMSReader(MaxQuantReader, PSMReader_w_FragBase):
    def __init__(self,
        frag_types=['b','y','b-modloss','y-modloss'], 
        max_frag_charge=2,
        score_threshold=100,
        **kwargs
    ):
        PSMReader_w_FragBase.__init__(self,
            frag_types = frag_types,
            max_frag_charge = max_frag_charge,
            **kwargs
        )

        MaxQuantReader.__init__(self)
        self._score_thres = score_threshold
    
    @property
    def fragment_intensity_df(self):
        return self._fragment_intensity_df

    def _load_file(self, filename):
        df = super()._load_file(filename)
        df = df[df.Score >= self._score_thres]
        df.reset_index(drop=True, inplace=True)
        return df

    def _post_process(self, 
        mq_df
    ):
        self._fragment_intensity_df = init_fragment_by_precursor_dataframe(
            mq_df, self.charged_frag_types
        )

        frag_col_dict = dict(zip(
            self.charged_frag_types, 
            range(len(self.charged_frag_types))
        ))

        for ith_psm, (nAA, start,end) in enumerate(
            mq_df[['nAA','frag_start_idx','frag_end_idx']].values
        ):
            intens = np.zeros((nAA-1, len(self.charged_frag_types)))

            frag_types = mq_df['Matches'].values[ith_psm]
            frag_intens = mq_df['Intensities'].values[ith_psm]
            for frag_type, frag_inten in zip(
                frag_types.split(';'), frag_intens.split(';')
            ):
                if '-' in frag_type: continue
                if any(_.isupper() for _ in frag_type): continue
                idx = frag_type.find('(')
                charge = '1'
                if idx > 0:
                    frag_type, charge = frag_type[:idx], frag_type[idx+1:-2]
                frag_type, frag_pos = frag_type[0], int(frag_type[1:].strip('*'))
                if frag_type in 'xyz':
                    frag_pos = nAA - frag_pos -1
                else:
                    frag_pos -= 1 
                frag_type += '_z'+charge
                if frag_type not in frag_col_dict: continue
                frag_col = frag_col_dict[frag_type]
                intens[frag_pos,frag_col] = float(frag_inten)

            if np.any(intens>0):
                intens /= np.max(intens)
            self._fragment_intensity_df.iloc[
                start:end,:
            ] = intens
        
        
        self._psm_df[
            ['frag_start_idx','frag_end_idx']
        ] = mq_df[['frag_start_idx','frag_end_idx']]

    def load_fragment_intensity_df(self, 
        psm_df, ms_files=None
    ):
        raise NotImplementedError(
            f'"{self.__class__}" must implement "load_fragment_intensity_df()"'
        )


psm_w_frag_reader_provider.register_reader('maxquant', MaxQuantMSMSReader)

In [20]:
#hide
import io
mq_tsv = io.StringIO('''Raw file	Scan number	Scan index	Sequence	Length	Missed cleavages	Modifications	Modified sequence	Oxidation (M) Probabilities	Oxidation (M) Score diffs	Acetyl (Protein N-term)	Oxidation (M)	Proteins	Charge	Fragmentation	Mass analyzer	Type	Scan event number	Isotope index	m/z	Mass	Mass error [ppm]	Mass error [Da]	Simple mass error [ppm]	Retention time	PEP	Score	Delta score	Score diff	Localization prob	Combinatorics	PIF	Fraction of total spectrum	Base peak fraction	Precursor full scan number	Precursor Intensity	Precursor apex fraction	Precursor apex offset	Precursor apex offset time	Matches	Intensities	Mass deviations [Da]	Mass deviations [ppm]	Masses	Number of matches	Intensity coverage	Peak coverage	Neutral loss level	ETD identification type	Reverse	All scores	All sequences	All modified sequences	Reporter PIF	Reporter fraction	id	Protein group IDs	Peptide ID	Mod. peptide ID	Evidence ID	Oxidation (M) site IDs
20190402_QX1_SeVW_MA_HeLa_500ng_LC11	81358	73979	AAAAAAAAAPAAAATAPTTAATTAATAAQ	29	0	Unmodified	_(Acetyl (Protein N-term))AAAAAAAAM(Oxidation (M))PAAAATAPTTAATTAATAAQ_			0	0	sp|P37108|SRP14_HUMAN	3	HCD	FTMS	MULTI-MSMS	13	1	790.07495	2367.203	0.35311	0.00027898	-0.061634807	70.261	0.012774	41.423	36.666	NaN	NaN	1	0	0	0	81345	10653955	0.0338597821787898	-11	0.139877319335938	y1;y2;y3;y4;y11;y1-NH3;y2-NH3;a2;b2;b3;b4;b5;b6;b7;b8;b9;b11;b12;b6(2+);b8(2+);b13(2+);b18(2+)	2000000;2000000;300000;400000;200000;1000000;400000;300000;600000;1000000;2000000;3000000;3000000;3000000;3000000;2000000;600000;500000;1000000;2000000;300000;200000	5.2861228709844E-06;-6.86980268369553E-05;-0.00238178789771837;0.000624715964988809;-0.0145624692099773;-0.000143471782706683;-0.000609501446461991;-0.000524972720768346;0.00010190530804266;5.8620815195809E-05;0.000229901232955854;-0.000108750048696038;-0.000229593152369034;0.00183148682538103;0.00276641182404092;0.000193118923334623;0.00200988580445483;0.000102216846016745;5.86208151389656E-05;0.000229901232955854;-0.00104559184393338;0.00525030008475369	0.0359413365445091;-0.314964433555295;-8.23711898839045;1.60102421155213;-14.8975999917227;-1.10320467763838;-3.03102462870716;-4.56152475051625;0.712219104095465;0.273777366204575;0.806231096969562;-0.305312183824154;-0.537399178230218;3.67572664689217;4.85930954169285;0.301587577451224;2.48616190909398;0.116225745519871;0.273777365939099;0.806231096969562;-2.19774169175011;7.53961026980589	147.076413378177;218.113601150127;289.153028027798;390.197699998035;977.50437775671;130.050013034583;201.087592852046;115.087114392821;143.081402136892;214.118559209185;285.155501716567;356.192954155649;427.230188786552;498.265241494374;569.301420357176;640.341107437877;808.429168310795;879.468189767554;214.118559209185;285.155501716567;475.757386711244;696.362265007215	22	0.262893575628735	0.0826446280991736	None	Unknown		41.4230894199432;4.75668724862449;3.9515580701967	AAAAAAAAAPAAAATAPTTAATTAATAAQ;FHRGPPDKDDMVSVTQILQGK;PVTLWITVTHMQADEVSVWR	_AAAAAAAAAPAAAATAPTTAATTAATAAQ_;_FHRGPPDKDDMVSVTQILQGK_;_PVTLWITVTHMQADEVSVWR_			0	1443	0	0	0	
20190402_QX1_SeVW_MA_HeLa_500ng_LC11	81391	74010	AAAAAAAAAPAAAATAPTTAATTAATAAQ	29	0	Unmodified	_AAAAAAAAAPAAAATAPTTAATTAATAAQ_			0	0	sp|P37108|SRP14_HUMAN	2	HCD	FTMS	MULTI-MSMS	14	0	1184.6088	2367.203	0.037108	4.3959E-05	1.7026696	70.287	7.1474E-09	118.21	100.52	NaN	NaN	1	0	0	0	81377	9347701	0.166790347889974	-10	0.12664794921875	y1;y2;y3;y4;y5;y9;y12;y13;y14;y20;y13-H2O;y20-H2O;y1-NH3;y20-NH3;b3;b4;b5;b6;b7;b8;b9;b11;b12;b13;b14;b15;b16;b19;b15-H2O;b16-H2O	500000;600000;200000;400000;200000;100000;200000;1000000;200000;300000;200000;100000;100000;70000;300000;900000;2000000;3000000;5000000;8000000;6000000;600000;800000;600000;200000;300000;200000;300000;300000;1000000	-0.000194444760495571;0.000149986878682284;0.000774202587820128;-0.0002445094036716;0.000374520568641401;-0.00694293246522193;-0.0109837291331587;-0.0037745820627606;-0.000945546471939451;0.00152326440706929;0.00506054832726477;0.00996886361417637;6.25847393393997E-05;-0.024881067836759;-3.11821549132674E-05;-0.000183099230639527;0.000161332473453513;0.000265434980121881;0.000747070697229901;0.000975534518261156;0.00101513939785036;0.00651913000274362;0.0058584595163893;0.00579536744021425;0.00131097834105276;-0.0131378531671089;0.00472955218901916;-0.00161006322559842;-0.00201443239325272;0.0227149399370319	-1.32206444236914;0.687655553213019;2.6775131607882;-0.626628140021726;0.811995006209331;-8.6203492854282;-10.1838066275079;-3.21078702288986;-0.758483069159249;0.881072738747222;4.37168212373889;5.82682888353564;0.481236695337485;-14.5343986203644;-0.145630261806375;-0.642102166533079;0.452935954800214;0.621293379181583;1.49934012872483;1.71355878380837;1.58531240493271;8.06399202403175;6.6614096214532;6.09718023739784;1.28333378040908;-11.7030234519348;3.96235146626144;-1.07856912288932;-1.82370619437775;19.3220953109188	147.07661310906;218.113382465221;289.149872037312;390.198569223404;461.235063981231;805.411965958065;1078.54847749073;1175.59403219566;1246.62831694787;1728.87474561429;1157.57463237897;1710.85573532879;130.049806978061;1711.87460084504;214.118649012155;285.155914717031;356.192684073126;427.22969375842;498.266325910503;569.303211234482;640.340285417402;808.424659066597;879.462433524883;950.49961040476;1021.54120858166;1122.60333588727;1193.62258226971;1492.77704268533;1104.58164778019;1175.59403219566	30	0.474003002083763	0.167630057803468	None	Unknown		118.209976573419;17.6937689289157;17.2534171481793	AAAAAAAAAPAAAATAPTTAATTAATAAQ;SELKQEAMQSEQLQSVLYLK;VGSSVPSKASELVVMGDHDAARR	_AAAAAAAAAPAAAATAPTTAATTAATAAQ_;_SELKQEAM(Oxidation (M))QSEQLQSVLYLK_;_VGSSVPSKASELVVMGDHDAARR_			1	1443	0	0	1	
20190402_QX1_SeVW_MA_HeLa_500ng_LC11	107307	98306	AAAAAAAGDSDSWDADAFSVEDPVRK	26	1	Acetyl (Protein N-term)	_(Acetyl (Protein N-term))AAAAAAAGDSDSWDADAFSVEDPVRK_			1	0	sp|O75822|EIF3J_HUMAN	3	HCD	FTMS	MULTI-MSMS	10	2	879.06841	2634.1834	-0.93926	-0.00082567	-3.2012471	90.978	2.1945E-12	148.95	141.24	NaN	NaN	1	0	0	0	107297	10193939	0.267970762043589	-8	0.10211181640625	y1;y2;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y15;y17;y18;y19;y20;y21;y23;y21-H2O;y1-NH3;y19-NH3;y14(2+);y16(2+);y22(2+);a2;b2;b3;b4;b5;b6;b7	300000;200000;3000000;600000;1000000;500000;2000000;1000000;1000000;1000000;90000;1000000;400000;900000;1000000;400000;3000000;2000000;1000000;400000;100000;200000;200000;80000;100000;200000;200000;2000000;5000000;5000000;5000000;2000000;300000	1.34859050149316E-07;-6.05140996867704E-06;2.27812602133781E-05;0.00128986659160546;-0.00934536073077652;0.000941953783126337;-0.00160424237344614;-0.00239257341399934;-0.00111053968612396;-0.00331340710044969;0.00330702864630439;0.000963683996815234;0.00596290290945944;-0.00662057038289277;-0.0117122701335575;0.00777853472800416;0.0021841542961738;0.000144322111736983;-0.00087403893667215;0.0197121595674616;-0.021204007680808;-0.000308954599830713;-0.026636719419912;-0.0137790992353075;0.00596067266928912;-0.0077053835773313;9.11402199221811E-06;-0.000142539300128419;-0.000251999832926231;1.90791054137662E-05;-0.00236430185879044;-9.54583337602344E-05;-0.000556959493223985	0.000916705048437201;-0.0199575598103408;0.0456231928690862;2.09952637717462;-12.5708704058425;1.11808305811426;-1.72590731777249;-2.22239181008062;-0.967696370445928;-2.62418809422166;2.47964286628144;0.665205752892023;3.64753748704453;-3.84510115530963;-6.08782672045773;3.81508105974837;1.04209904973991;0.0666012719936656;-0.390545453668809;8.28224925531311;-9.55133250134922;-2.37499239179248;-12.8127653858411;-16.846761946123;6.48662354975264;-6.67117082062383;0.0580151981289049;-0.770098855873447;-0.983876895688683;0.0583162347158579;-5.93738717724506;-0.203431522818505;-1.03087538746314	147.112804035741;303.21392125011;499.33507018564;614.360746132308;743.413974455831;842.472101057517;929.506675663573;1076.57587791081;1147.61170966489;1262.6408555643;1333.67134891635;1448.700635293;1634.77494902759;1721.81956091078;1923.88362405243;2038.89107627957;2095.9181343836;2166.95728800359;2237.99542015244;2380.04906152953;2220.00518543488;130.0865640237;2078.92040615582;817.907873297785;918.917619246831;1155.02717356753;157.097144992378;185.0922112678;256.129434516133;327.166277224995;398.205774393759;469.240619338034;540.278194626993	33	0.574496146107112	0.14410480349345	None	Unknown		148.951235201399;7.71201258444522;7.36039532447559	AAAAAAAGDSDSWDADAFSVEDPVRK;PSRQESELMWQWVDQRSDGER;HTLTSFWNFKAGCEEKCYSNR	_(Acetyl (Protein N-term))AAAAAAAGDSDSWDADAFSVEDPVRK_;_PSRQESELM(Oxidation (M))WQWVDQRSDGER_;_HTLTSFWNFKAGCEEKCYSNR_			2	625	1	1	2	'''
)

mq_reader = psm_w_frag_reader_provider.get_reader('maxquant')
mq_reader.import_file(mq_tsv)
assert 'frag_start_idx' in mq_reader.psm_df.columns
assert 'frag_end_idx' in mq_reader.psm_df.columns
assert len(mq_reader.psm_df) == 2
assert mq_reader.psm_df.mods.values[0] == ''
assert mq_reader.psm_df.mod_sites.values[0] == ''
assert mq_reader.psm_df.mods.values[1] == 'Acetyl@Protein N-term'
assert mq_reader.psm_df.mod_sites.values[1] == '0'
seq = 'AAAAAAAGDSDSWDADAFSVEDPVRK'
frag_types = 'y1;y2;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y15;y17;y18;y19;y20;y21;y23;y21-H2O;y1-NH3;y19-NH3;y14(2+);y16(2+);y22(2+);a2;b2;b3;b4;b5;b6;b7'.split(';')
frag_intens = '300000;200000;3000000;600000;1000000;500000;2000000;1000000;1000000;1000000;90000;1000000;400000;900000;1000000;400000;3000000;2000000;1000000;400000;100000;200000;200000;80000;100000;200000;200000;2000000;5000000;5000000;5000000;2000000;300000'.split(';')
frag_intens = np.array([float(frag_intens[i]) for i,frag_type in enumerate(frag_types) if '-' not in frag_type and frag_type[0] in 'by'])
frag_types = [frag_type for frag_type in frag_types if '-' not in frag_type and frag_type[0] in 'by']
frag_intens /= np.max(frag_intens)
df = pd.DataFrame(
    {'frag':frag_types, 'inten':frag_intens}
)

def query(frag_type, intens, nAA):
    if frag_type[0] == 'y':
        if frag_type[-1].isdigit():
            return intens[nAA-int(frag_type[1:])-1,2]
        else:
            return intens[nAA-int(frag_type[1:-4])-1,3]
    else:
        if frag_type[-1].isdigit():
            return intens[int(frag_type[1:])-1,0]
        else:
            return intens[int(frag_type[1:-4])-1,1]

start,end = (mq_reader.psm_df[mq_reader.psm_df.sequence==seq])[['frag_start_idx','frag_end_idx']].values[0]
intens = mq_reader.fragment_intensity_df.values[start:end,:]
for frag_type, frag_inten in zip(frag_types, frag_intens):
    assert abs(query(frag_type, intens, len(seq))-frag_inten) < 1e-5, (
        display(mq_reader.fragment_intensity_df.iloc[start:end,:]),
        display(df),
        frag_type, frag_inten
    )


In [4]:
#hide
mq_reader.psm_df

,sequence,charge,rt,spec_idx,raw_name,precursor_mz,score,proteins,decoy,nAA,mods,mod_sites,frag_start_idx,frag_end_idx
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,2,70.287,81391,20190402_QX1_SeVW_MA_HeLa_500ng_LC11,1184.60880,118.21,sp|P37108|SRP14_HUMAN,0,29,,,0,28
1,AAAAAAAGDSDSWDADAFSVEDPVRK,3,90.978,107307,20190402_QX1_SeVW_MA_HeLa_500ng_LC11,879.06841,148.95,sp|O75822|EIF3J_HUMAN,0,26,Acetyl@Protein N-term,0,28,53


In [6]:
#hide
mq_reader.fragment_intensity_df

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,0.0000,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
1,0.0000,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
2,0.0375,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
3,0.1125,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
4,0.2500,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
5,0.3750,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
6,0.6250,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
7,1.0000,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
8,0.7500,0.0,0.0375,0.000,0.0,0.0,0.0,0.0
9,0.0000,0.0,0.0000,0.000,0.0,0.0,0.0,0.0
